# Data Treatment

Rui

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#goes to the previous folder, accesses the data folder, and gets the path to the donors.csv file (this is done bc the is
#too large to upload on GitHub)
path = os.path.join('..', 'data', 'donors.csv')

donors = pd.read_csv(path, sep = ',')

pd.set_option('display.max_columns', None)

C:\Users\ruifi\anaconda3\envs\DMclasses\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#CONTROLN: stays as index of the dataset
donors.set_index('CONTROLN', inplace=True)

In [4]:
#put aside the variables that are already related to a PVA's clustering, and may be used in the end to compare it to our
#solution...:
donors_pva_cluster = donors[['MDMAUD', 'DOMAIN', 'GEOCODE', 'PEPSTRFL']].copy()

#MDMAUD: RFA field, already a clustering from the organization

#put this aside because it is clustering (this one is a bit different from the RFA variables)
#donors_pva_cluster.replace({'DOMAIN' : {' ' : np.nan}}, inplace = True)

#GEOCODE is what? Put aside for now - clustering
#donors_pva_cluster.replace({'GEOCODE' : {' ' : np.nan}}, inplace = True)

#probably put aside! bc it's related to RFA
#donors_pva_cluster.replace({'PEPSTRFL' : {'X' : '1', ' ' : '0'}}, inplace = True)


rfa1 = donors.iloc[:, 382:405].copy()
donors_pva_cluster.append(rfa1)
#from RFA_2 to RFA_24: these are variables with a clustering solution from PVA; don't use them for now (maybe use them
#in the end to compare with our own clustering solution)

ramnt1 = donors.iloc[:, 432:454].copy()
donors_pva_cluster.append(ramnt1)
#from RAMNT_3 until RAMNT_24 -> explicado na parte do Tomás

rfa_and_mdmaud = donors.iloc[:, 468:474].copy()
donors_pva_cluster.append(rfa_and_mdmaud)
#RFA_2R, RFA_2F, RFA_2A are related to RFA (put aside for now?)
#MDMAUD_R, MDMAUD_F, MDMAUD_A are related to RFA (put aside for now?)

,MDMAUD,DOMAIN,GEOCODE,PEPSTRFL
CONTROLN,,,,
95515,XXXX,T2,,X
148535,XXXX,S1,02,
15078,XXXX,R2,,X
172556,XXXX,R2,,X
7112,XXXX,S2,,
...,...,...,...,...
184568,XXXX,C2,,
122706,XXXX,C1,,
189641,XXXX,C3,,X


In [7]:
#drop these:
donors.drop(columns = ['Unnamed: 0',
                       'NOEXCH',
                       'WEALTH1',
                       'DATASRCE',
                       'LIFESRC'], inplace = True)

#Unnamed: 0: it has indexes from 0 to 95412, it's redundant

#NOEXCH: this var. has inconsistencies with the metadata available, as there's no reference to '0', '1', 0 or 1 values
#BELO - COHERENCE
#drop this! (incoherent, bc the variable has many incoherences with the metadata -> explicar melhor no relatório)
#donors['NOEXCH'].value_counts()

#WEALTH1 has similar values to WEALTH2 but the metadata does not specify what is the variable, so we decided to keep
#the WEALTH2, which is well described
#donors = donors.astype({"WEALTH1": object})
#donors['WEALTH1'].unique()

#donors['DATASRCE'].unique()
#donors.replace({'DATASRCE' : {' ' : np.nan}}, inplace = True)
#says which was the source of the data in this section of variables; it has 21280 ' ' (blanks)
#DROP! bc the variable is not very useful for interpreting the clusters (we are not sure about what are the values 
#representing... companies? we could not find on the internet)

#LIFESRC: same as DATASRCE...
#donors['LIFESRC'].value_counts()
#donors.replace({'LIFESRC' : {' ' : np.nan}}, inplace = True)
#says which was the source of the data in this section of variables; it has 54032 ' ' (blanks)
#DROP! bc the variable is not very useful for interpreting the clusters (we are not sure about what are the values 
#representing... companies? we could not find on the internet)

In [ ]:
#drop variables on the extra dataframe that has the clustering from PVA (donors_pva_cluster)

#CODE HERE!

In [5]:
#note -> 'NUMCHLD' : int, altough it cannot be converted due to NaNs (float)!
donors = donors.astype({'TCODE': object, 
                        'INCOME': object,
                        'WEALTH2': object,
                        'MSA': object,
                        'ADI': object,
                        'DMA': object,
                        'HPHONE_D': object})

#INCOME: from the Forums: 7 correspond to the highest income group and 0 to the lowest one

#MSA has values 0, which is not a MSA code. After researching what does this code represent in the US census, we assume 
#those 0s are for the doners that do not live in a metropolitan statistical area (MSA), a region with at least one 
#urbanized area and a minimum population of 50 000 (Investopedia); so those people may live in towns or rural areas.

#ADI has values 0, which is not an ADI code. After researching what does this code represents, we assume 
#those 0s are for the doners that do not live in a radio or television market region in which that specific population 
#receives the same programming (https://www.insightsassociation.org/issues-policies/glossary/area-dominant-influence-adi);
#this is related to marketing, and those people may live in regions that are not assigned to a code (but this is not a 
#missing value!).

#DMA has values 0, which is not a DMA code. After researching what does this code represents, we assume 
#those 0s are for the doners that do not live in a geographic area in the United States in which local television viewing 
#is measured by Nielsen (https://www.nielsen.com/us/en/intl-campaigns/dma-maps/#:~:text=DMA%20(Designated%20Market%20Area)%20regions,geographic%20areas%20within%20their%20business.)
#again, this code is related to marketing, and those people may live in regions that are not assigned to a code (but this 
#is not a missing value!).

#HPHONE_D is binary (0 or 1)

In [6]:
#dates as string, now as datetimes
#note -> errors='coerce' as in the vast majority of records (if not always) the errors on the to_datetime function are
#rased because the string in the column is empty, thus there isn't a date there; this leads to an error, and with this
#argument definition, the invalid parsing will be set as NaT (Not a Time), which is what we intend (missing values stay
#missing)
donors['ODATEDW'] = pd.to_datetime(donors['ODATEDW'])
donors['DOB'] = pd.to_datetime(donors['DOB'], errors='coerce')
donors.iloc[:, 359:382] = donors.iloc[:, 359:382].apply(pd.to_datetime, errors='coerce') #from ADATE_2 until ADATE_24
donors['MAXADATE'] = pd.to_datetime(donors['MAXADATE'])
donors.iloc[:, 410:432] = donors.iloc[:, 410:432].apply(pd.to_datetime, errors='coerce') #from RDATE_3 until RDATE_24
donors['MINRDATE'] = pd.to_datetime(donors['MINRDATE'])
donors['MAXRDATE'] = pd.to_datetime(donors['MAXRDATE'])
donors['LASTDATE'] = pd.to_datetime(donors['LASTDATE'])
donors['FISTDATE'] = pd.to_datetime(donors['FISTDATE'], errors='coerce')
donors['NEXTDATE'] = pd.to_datetime(donors['NEXTDATE'], errors='coerce')

In [14]:
#to remove the hyphen from the end of the ZIP codes (US zip codes are not supposed to have hyphens, but some do!)
donors['ZIP'] = donors['ZIP'].map(lambda x: x.replace('-', ''))

#now, the replace: on some vars. swap a blank space for a NaN, on binaries swap the letters for '0' and '1', etc
donors.replace({'OSOURCE' : {' ' : np.nan},
                'MAILCODE' : {' ' : '0', 'B' : '1'},
                'PVASTATE' : {' ' : 'N'}, #N = state is not served by any organization
                'RECINHSE' : {' ' : '0', 'X' : '1'},
                'RECP3' : {' ' : '0', 'X' : '1'},
                'RECPGVG': {' ' : '0', 'X' : '1'},
                'RECSWEEP': {' ' : '0', 'X' : '1'},
                'HOMEOWNR' : {' ' : '0', 'H' : '1', 'U' : '0'}, #we assume the blank space on this variable says that it is unknown wether the donor has or not a home (so, we joined it with the 0-Unkwown)
                'CHILD03' : {' ' : 'N'}, #the ' ' (blank) will be replaced by 'N' (N = None), as we assume the blank space in this variable means the donor has no 
                'CHILD07' : {' ' : 'N'}, #children in that specific range of ages
                'CHILD12': {' ' : 'N'},  #note: more than 90 000 records have None, on each of these 4 variables
                'CHILD18': {' ' : 'N'},
                'GENDER' : {' ' : 'U', 'A' : 'U', 'C' : 'U'}, #Incoherence (Belo!) -> nem que se fale só no relatório!
                                                             #values 'A' (2 of them) & 'C' (2) are not described on the metadata, most probably those are errors
                                                             #we assume the blank spaces, As and Cs can be assumed to be Unkwown
                'SOLP3' : {' ' : 'Default'},
                'SOLIH' : {' ' : 'Default'}, #feature engineering: create a binary with -> 0 (can't be mailed), 1 (can be mailed)
                'MAJOR' : {' ' : '0', 'X' : '1'},
                'GEOCODE2' : {' ' : np.nan}}, inplace = True)

#assuming Y = Yes and N = No, on all these vars
donors.iloc[:, 53:71] = donors.iloc[:, 53:71].replace({'Y' : '1', ' ' : '0'}) #from COLLECT1 to PLATES

In [9]:
#PROBLEM on HHP1 and HHP2: "Median Person Per Household" and "Average Person Per Household" but the values are bery high
#if we interpret this as the median/average number of people living in a household of the neighborhood, then it should
#have values between 1, 2 or 3. Not 100-700. (Incoherence - Belo?)



#perguntar ao professor!!
#decisão final: (nota: ainda há alguns valores iguais a 0)
donors['HHP1'] = donors['HHP1'] / 100
donors['HHP2'] = donors['HHP2'] / 100

In [10]:
#RHP1 to RHP4 are kinda weird (Housing Unit??)



#perguntar ao professor: média/mediana das pessoas ser tão alto (50?); ou a descrição da variável não está correta?
#decisão final: we assume this values were multiplied by 10 to avoid storing floats (floats make up more space on data
#files), so we divided this columns by 10, for the values to make more "sense"
donors['RHP1'] = donors['RHP1'] / 10
donors['RHP2'] = donors['RHP2'] / 10
donors['RHP3'] = donors['RHP3'] / 10
donors['RHP4'] = donors['RHP4'] / 10

In [11]:
#EC1: "Median Years of School Completed by Adults 25+" has minimum 89 and maximum 170... are the values wrong?

#HC1: "Percent Median Length of Residence" is percent and median at the same time? And has values of 0 length! Is it the 
#median time someone has been living in their current home, in years? Very ambiguous:
#HC2: this one has a less ambiguous description but it still says "Percent Median" so is it a percentage or a median? 
#it is confusing


#perguntar ao professor: EC1 (mediana de 170 anos a estudar?):
#decisão final: divided by 10, because 8.9 (min) and 17.0 (max) are values that make more sense
#we assume this values were multiplied by 10 to avoid storing floats (floats make up more space on data
#files), so we divided this columns by 10, for the values to make more "sense"
donors['EC1'] = donors['EC1'] / 10
#HC1 e HC2 -> we assume this is a median, the "percent" is probably a typo on the metadata descriptions

In [12]:
#BELO - COHERENCE
#it has more than 50 states bc it includes US territories like the Virgin Islands, Northern Mariana Islands, Guam, etc
donors['STATE'].nunique()

57

In [22]:
donors.head()

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
CONTROLN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
95515,0,2009-01-01,GRI,0,IL,61081,0,N,1957-12-01,0,0,0,0,0,XXXX,T2,0,N,N,N,N,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,Default,Default,0,5,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,1.98,2.76,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,

In [24]:
#'HIT' is good to go

In [25]:
#from MBCRAFT to PUBOPP, it all looks good; but it has many missing values!

In [27]:
#from MALEMILI to FEDGOV there are no MVs and everything is fine (check for incoherences: percentages below 0 or 
#above 100?)

In [35]:
#from POP901 until the end of this set of variables, we assume 0s in the first three (Number of Persons, Families and 
#Households) represent people that live in rural areas and small towns, with no one living near them (in the neighborhood)

#gets resolved when we do donors = donors[donors['AGE902'] != 0] (Coherence checks!)

In [127]:
#from POP901 until ETH16, everything looks fine

In [45]:
#PROBLEM on AGE902: "Median Age of Adults 18 or Older" but it had a value that was 0... actually, after analyzing the 
#subset of donors that had AGE902 equal to 0, we found out that the vast majority of them had also 0 in basically all
#socio-demographic variables. This leads tu belive the missing values on this columns may be represented by a 0 other than
#a Numpy NaN. Anyways, these records have incoherences, and for this reason, they should be removed (799 records!).

#They have donations... but they should be removed anyways.
#dps disto, o MSA, ADI e DMA deixam de ter missings!

donors = donors[donors['AGE902'] != 0]

In [68]:
#PROBLEM on HV1 & HV2: "Median Home Value in hundreds" has values of 0, and then, the second lowest, 75 (which means 7500) 
#the values of 0 are 318 records - same on HV2



#perguntar ao professor: nestas variáveis, assumimos que os 0s são missings, incoerências, ou representam casas alugadas.
#decisão final:
#assumption -> there were homes with very low market values, so when the values where divided by 100 to get the value for 
#the column, it was so close to 0 that the int was 0

In [69]:
#HV3 and HV4, related to rents, also have values 0 but that can mean the people are not paying any rent (maybe they bought
#the house already, or maybe they live with family members and are not paying, etc)

In [ ]:
#IC1-IC5: values 0 -> don't declare anything to the state?

In [67]:
#donors['TPE10'] has values of 0: people that work at home or live very near to the company/office/...

In [101]:
#RFA_2R, RFA_2F, RFA_2A are related to RFA (put aside for now?)
#MDMAUD_R, MDMAUD_F, MDMAUD_A are related to RFA (put aside for now?)